### Ежемесячная выгрузка на основе UTM размеченных ссылок
Скрипт забирает по АПИ Яндекс Метрики данные для списка сайтов, размеченные определенными UTM метками и сводит в единую экслеь таблицу определенного формата. 
Часть данных нестандартизированна, поэтому обрабатывается отдельными запросами вне цикла. 
Скрипт частично отредактирован, так как содержит данные заказчика.

In [1]:
import pandas as pd
import requests
import json

startDate = '2021-02-01' 
endDate = '2021-02-28'

projects = [ # список списков формата ["название сайта", "номер счетчика"]
                ]

In [2]:
def collect_data(data, utm_source):
    """
    Цель - собрать данные по UTM метке для ответа из АПИ для конкретного счетчика
    """
    new_list = []
    
    for counter in data:
        n = 0
        try:
            for metric in counter['data']:
                if {'name': utm_source} in metric['dimensions']:
                    n = metric['metrics'][0]
        except: n = 9999 # в случае ошибки ответа срипта, значение закрывается болванкой
        new_list.append(n)
    return new_list


# UTM Capmpaign = XXXXX


In [3]:
# UTM Capmpaign = COMMUNITITES_1MI

api_response = []
for project, counter in projects:

    # ТЕСТОВЫЙ ЗАПРОС В API ЯНДЕКС МЕТРИКА
    header = {'Authorization': #TOKEN }
    payload = {
        'metrics': 'ym:s:visits',
        'dimensions': ['ym:s:UTMCampaign','ym:s:UTMSource'],
        'date1': startDate,
        'date2': endDate,
        'accuracy': '1',
        'attribution': 'last',
        'filters': "ym:s:UTMCampaign=='XXXXX'",
        'ids': counter
        }

    r = requests.get('https://api-metrika.yandex.ru/stat/v1/data', params=payload, headers=header)
    response = json.loads(r.content)
    api_response.append(response)
    
    if 'totals' not in response.keys():
        print(project, counter, '\n', response)

In [ ]:
df = pd.DataFrame({#'Сайт': [counter[0] for counter in projects],
            'Все группы в соцсетях*': [counter['totals'][0] if 'totals' in counter.keys() else 9999 for counter in api_response],
            'Вконтакте': collect_data(api_response, 'vk.com'),
            'Facebook': collect_data(api_response, 'facebook.com'),
            'Одноклассники': collect_data(api_response, 'ok.ru'),
            'Twitter': collect_data(api_response, 'twitter.com'),
            'Instagram': collect_data(api_response, 'instagram.com'),
            'Telegram': collect_data(api_response, 'telegram'), 
            'Яндекс.Район': collect_data(api_response, 'yandex_rayon')
             }, index=[counter[0] for counter in projects])

df['Все группы в соцсетях*'] = df['Все группы в соцсетях*'] - df['Яндекс.Район']

payload = { #'ААААА' }
r = requests.get('https://api-metrika.yandex.ru/stat/v1/data', params=payload, headers=header)
response = json.loads(r.content)
aaaaa = response['totals'][0]

payload = { #'БББББ' }    
r = requests.get('https://api-metrika.yandex.ru/stat/v1/data', params=payload, headers=header)
response = json.loads(r.content)
bbbbb = response['totals'][0]

extra_gorod = pd.DataFrame({
            'Все группы в соцсетях*': [None, None],
            'Вконтакте': [None, None],
            'Facebook': [None, None],
            'Одноклассники': [aaaaa, bbbbb],
            'Twitter': [None, None],
            'Instagram': [None, None],
            'Telegram': [None, None], 
            'Яндекс.Район': [None, None]
             }, index=['ААААА', 'БББББ'])

df = pd.concat([df, extra_gorod], ignore_index=False)

df = df.reindex( # СПИСОК НАИМЕНОВАНИЯ САЙТОВ )

df.to_excel(f'UTM.xlsx')
df